The idea is to compute a genetic distance between two species. This will be used in comparing the genomic FGOC 
score between two species. <br />
I am using a tree computed (from the living tree of life) and downloaded from [silva DB](http://www.arb-silva.de/projects/living-tree/). This contains a curated list of 16sRNA and 23sRNA from bacterial and archael species. The tree nodes have been collapsed to the species level. This means I will have to use the same distance metric when comparing it at a strain level. <br />
The metric for the distance between two species is going to be a patristic distance. I will be calculating the distance matrix from this tree. The computation is intensive and requires a lot of memory. The file used is calcGeneticIt was stored as a massive flatfile in the format - BACTERIA-A BACTERIA-B distance <br />



In [24]:
import os
import pickle 

def loadPkl(fname):
    pklDir = '/home/jaggu/research/projectFiles/operons/pklFiles'
    f = os.path.join(pklDir,fname)
    db = pickle.load(open(f))
    return db

cogPair_dict = loadPkl('cogPair_count.dict.pkl')
cog_locusTag_dict = loadPkl('cog_locusTag.dict.pkl')
locusTag_cog_dict = loadPkl('locus_cog.dict.pkl')
org_cogPair_dict = loadPkl('org_cogPair.dict.pkl')

In [22]:
# Making a dictionary for (Org1,org2):dist where Org1 is either Escherichia or Bacillus etc.
import os
import cPickle as pickle

def savePkl(db,pklFname):
    pklDir = '/home/jaggu/research/projectFiles/operons/pklFiles'    
    f = os.path.join(pklDir,pklFname)
    pickle.dump(db,open(f,'w'))
    return 

bactTreeDir = '/home/jaggu/research/downloads/bacterialTrees/silvaDB'
distFile = 'all.bactDist.txt'
fname = os.path.join(bactTreeDir,distFile)
#org = 'eco'
#orgName = 'Escherichia coli'
org = 'bsu'
orgName = 'Bacillus subtilis'
orgDistDict = dict()

with open(fname) as f:
    for line in f:
        if not line.startswith('BACTERIA'):
            if line.startswith(orgName):
                newLine = line.replace(' ','_')
                org1,org2,dist = newLine.strip().split('\t')
                dist = float(dist)
                org1Name = org1.split('_')[0]+'_'+org1.split('_')[1]
                org2Name = org2.split('_')[0]+'_'+org2.split('_')[1]
                #print org1Name, org2Name, dist
                orgDistDict[org2Name]=dist
savePkl(orgDistDict,org+'_org2_dist.dict.pkl')
print org + " : Pickled"


bsuPickled


In [23]:
# Calculating genFGOC score by comparing gene pair between operonic genes and non-operonic genes. 

import os
import cPickle as pickle

operondbDir = '/home/jaggu/research/downloads/operons_databases/microbesOnline'
org = 'eco' 
#org = 'bsu'
fname = org+'.'+'operon'
ifile = open(os.path.join(operondbDir,fname))
lines = ifile.readlines()
ifile.close()

wOp_COG_list = list()
bwOp_COG_list = list()

def getCog(lTag):
    try:
        cog = locus_cog_dict[lTag]
    except KeyError:
        cog = 'nan'
    if cog == '-': cog = 'nan' #Some wierd bug
    return cog

def getCOGpairInfo(lTag1,lTag2):
    cogA = getCog(lTag1)
    cogB = getCog(lTag2)
    if cogA is 'nan' or cogB is 'nan':
        return None
    else:
        try: 
            fgocInfo = cogPair_fgocInfo_dict[(cogA,cogB)]
            cogPair = (cogA,cogB)
        except KeyError: 
            fgocInfo = cogPair_fgocInfo_dict[(cogB,cogA)] #Opposite strand; I can find it this way
            cogPair = (cogB, cogA)
        return cogPair

for line in lines[1:]:
    lTag1,lTag2,bOp = [line.split('\t')[i] for i in [2,3,6]]
    
    
    fgocInfo = getFgocInfo(lTag1,lTag2)
    if fgocInfo:
        dirFgoc,fgoc = fgocInfo[3],fgocInfo[4]
        if bOp == 'TRUE':
            wOp_dict[(lTag1,lTag2)]=[dirFgoc,fgoc]
        else:
            bwOp_dict[(lTag1,lTag2)]=[dirFgoc,fgoc]




14146	14147	b0001	b0002	thrL	thrA	FALSE	0.322	82.000	0.043	1.000		0.395

